In [1]:
"""
Funciones de actualización de datos diarios para que solo se carguen los datos nuevo o actualizados.

Hay que actualizar la fecha de los ficheros .csv y ejecutar los scripts.
"""

'\nFunciones de actualización de datos diarios para que solo se carguen los datos nuevo o actualizados.\n\nHay que actualizar la fecha de los ficheros .csv y ejecutar los scripts.\n'

In [2]:
import pandas as pd
import mysql
import mysql.connector
import numpy as np

In [3]:
#función para actualizar tabla stocks
data = pd.read_csv("stocks_20241031.csv", index_col=0).replace({np.nan: None}).values
database = "yfinance_stocks"
table = "stocks"

def insert_to_stocks(data, table, database, host="localhost", user="root", password="Hackaboss_2024"):
    db = mysql.connector.connect(host=host, user=user, password=password, database=database)
    cursor = db.cursor()

    #consultas de inserción y actualización separadas
    insert_query = f"""
        INSERT INTO {table} (ticker, name, sector, industry, market_cap, full_time_employees, ipo_date, extraction_timestamp) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    update_query = f"""
        UPDATE {table} SET 
            name = %s,
            sector = %s,
            industry = %s,
            market_cap = %s,
            full_time_employees = %s,
            ipo_date = %s,
            extraction_timestamp = %s
        WHERE ticker = %s
    """

    #obtener registros existentes
    existing_records = {}
    cursor.execute(f"SELECT ticker, name, sector, industry, market_cap, full_time_employees, ipo_date, extraction_timestamp FROM {table}")
    for row in cursor.fetchall():
        existing_records[row[0]] = row[1:]  #ticker : (name, sector, industry, market_cap, full_time_employees, ipo_date, extraction_timestamp)

    #filtrado de nuevos registros
    values_to_insert = []
    values_to_update = []
    for row in data:
        if len(row) != 8:  #que cada fila tenga 8 elementos
            continue
    
        ticker = row[0]  #campo ticker
        if ticker not in existing_records:
            values_to_insert.append(tuple(row))
        else:
            #verificar si hay cambios en los otros campos
            existing_values = existing_records[ticker]
            if existing_values != tuple(row[1:]):  #comparar todos los valores excepto ticker
                #añadir los campos de actualización seguidos por el ticker para WHERE
                values_to_update.append(tuple(row[1:]) + (ticker,))
    
    #totales
    rows_inserted = 0
    rows_updated = 0
    
    #insertar solo los nuevos registros
    if values_to_insert:
        cursor.executemany(insert_query, values_to_insert)
        rows_inserted = cursor.rowcount  
    
    #actualizar registros existentes que han cambiado
    if values_to_update:
        cursor.executemany(update_query, values_to_update)
        rows_updated = cursor.rowcount  
    
    db.commit()
    
    #resultados finales
    print(f"Total de registros existentes: {len(existing_records)}")
    print(f"Añadidas: {rows_inserted} filas nuevas")
    print(f"Actualizadas: {rows_updated} filas")
    
    cursor.close()
    db.close()

insert_to_stocks(data=data, table="stocks", database="yfinance_stocks")

Total de registros existentes: 57
Añadidas: 0 filas nuevas
Actualizadas: 57 filas


In [4]:
#función para actualizar tabla historical_prices
data = pd.read_csv("historical_prices_20241031.csv").replace({np.nan: None}).values
database = "yfinance_stocks"
table = "historical_prices"

def insert_to_historical_prices(data, table, database, host="localhost", user="root", password="Hackaboss_2024"):
    db = mysql.connector.connect(host=host, user=user, password=password, database=database)
    cursor = db.cursor()

    #consulta de inserción con actualización en caso de duplicados
    insert_query = f"""
        INSERT INTO {table} (date, open, high, low, close, adj_close, volume, ticker, extraction_timestamp) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            open = IF(VALUES(open) != open, VALUES(open), open),
            high = IF(VALUES(high) != high, VALUES(high), high),
            low = IF(VALUES(low) != low, VALUES(low), low),
            close = IF(VALUES(close) != close, VALUES(close), close),
            adj_close = IF(VALUES(adj_close) != adj_close, VALUES(adj_close), adj_close),
            volume = IF(VALUES(volume) != volume, VALUES(volume), volume),
            extraction_timestamp = CASE 
                WHEN VALUES(open) != open OR 
                     VALUES(high) != high OR 
                     VALUES(low) != low OR 
                     VALUES(close) != close OR 
                     VALUES(adj_close) != adj_close OR 
                     VALUES(volume) != volume 
                THEN VALUES(extraction_timestamp) 
                ELSE extraction_timestamp 
            END
    """

    #obtener registros existentes
    existing_records = {}
    cursor.execute(f"SELECT date, ticker, open, high, low, close, adj_close, volume, extraction_timestamp FROM {table}")
    for row in cursor.fetchall():
        existing_records[(row[0], row[1])] = row[2:]  #(date, ticker) : (open, high, low, close, adj_close, volume, extraction_timestamp)

    #filtrado de nuevos registros
    values_to_insert = []
    records_to_update = []
    for row in data:
        record = (row[0], row[7])  #campos date y ticker
        if record not in existing_records:
            values_to_insert.append(tuple(row))
        else:
            #verificar si hay cambios en los otros campos
            existing_values = existing_records[record]
            if existing_values != tuple(row[1:7]):  #comparar los valores de open, high, low, close, adj_close, volume
                records_to_update.append(tuple(row))

    #totales
    rows_inserted = 0
    rows_updated = 0

    #insertar solo los nuevos registros    
    if values_to_insert:
        cursor.executemany(insert_query, values_to_insert)
        rows_inserted = cursor.rowcount

    #actualizar registros existentes que han cambiado
    if records_to_update:
        cursor.executemany(insert_query, records_to_update)
        rows_updated = cursor.rowcount
        
    db.commit()

    #resultados finales
    print(f"Total de registros existentes: {len(existing_records)}")
    print(f"Añadidas: {rows_inserted} filas nuevas")
    print(f"Actualizadas: {rows_updated} filas")
    
    cursor.close()
    db.close()

insert_to_historical_prices(data=data, table=table, database=database)

Total de registros existentes: 40849
Añadidas: 2464 filas nuevas
Actualizadas: 0 filas


In [5]:
#función para actualizar tabla dividends
data = pd.read_csv("dividends_20241031.csv").replace({np.nan: None}).values
database = "yfinance_stocks"
table = "dividends"

def insert_to_dividends(data, table, database, host="localhost", user="root", password="Hackaboss_2024"):
    db = mysql.connector.connect(host=host, user=user, password=password, database=database)
    cursor = db.cursor()

    #consulta de inserción
    insert_query = f"""
        INSERT INTO {table} (date, dividends, ticker, extraction_timestamp) 
        VALUES (%s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            dividends = IF(VALUES(dividends) != dividends, VALUES(dividends), dividends),
            extraction_timestamp = CASE 
                WHEN VALUES(dividends) != dividends THEN VALUES(extraction_timestamp)
                ELSE extraction_timestamp
            END
    """

    #obtener registros existentes
    existing_records = {}
    cursor.execute(f"SELECT date, ticker, dividends, extraction_timestamp FROM {table}")
    for row in cursor.fetchall():
        existing_records[(row[0], row[1])] = row[2:]  #(date, ticker) : (dividends, extraction_timestamp)

    #filtrado de nuevos registros
    values_to_insert = []
    records_to_update = []
    for row in data:
        record = (row[0], row[2])  #campos date y ticker
        if record not in existing_records:
            values_to_insert.append(tuple(row))
        else:
            #verificar si hay cambios en el campo "dividends"
            existing_dividends = existing_records[record][0]
            if existing_dividends != row[1]:  #comparar el valor de dividends
                records_to_update.append(tuple(row))

    #totales
    rows_inserted = 0
    rows_updated = 0

    if values_to_insert:
        # Ejecutar la inserción de nuevos registros
        cursor.executemany(insert_query, values_to_insert)
        rows_inserted = cursor.rowcount

    if records_to_update:
        # Ejecutar actualización de registros existentes
        cursor.executemany(insert_query, records_to_update)
        rows_updated = cursor.rowcount

    db.commit()

    #resultados finales
    print(f"Total de registros existentes: {len(existing_records)}")
    print(f"Añadidas: {rows_inserted} filas nuevas")
    print(f"Actualizadas: {rows_updated} filas")
    
    cursor.close()
    db.close()

# Ejecutar la función
insert_to_dividends(data=data, table=table, database=database)

Total de registros existentes: 333
Añadidas: 0 filas nuevas
Actualizadas: 0 filas


In [6]:
#función para actualizar tabla splits
data = pd.read_csv("splits_20241031.csv").replace({np.nan: None}).values
database = "yfinance_stocks"
table = "splits"

def insert_to_splits(data, table, database, host="localhost", user="root", password="Hackaboss_2024"):
    db = mysql.connector.connect(host=host, user=user, password=password, database=database)
    cursor = db.cursor()

    #consultas de inserción
    insert_query = f"""
        INSERT INTO {table} (date, stock_splits, ticker, extraction_timestamp) 
        VALUES (%s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            stock_splits = IF(VALUES(stock_splits) != stock_splits, VALUES(stock_splits), stock_splits),
            extraction_timestamp = CASE 
                WHEN VALUES(stock_splits) != stock_splits THEN VALUES(extraction_timestamp)
                ELSE extraction_timestamp
            END
    """

    #obtener registros existentes
    existing_records = {}
    cursor.execute(f"SELECT date, ticker, stock_splits, extraction_timestamp FROM {table}")
    for row in cursor.fetchall():
        existing_records[(row[0], row[1])] = row[2:]  #(date, ticker) : (stock_splits, extraction_timestamp)

    #filtrado de nuevos registros
    values_to_insert = []
    records_to_update = []
    for row in data:
        record = (row[0], row[2])  #campos date y ticker
        if record not in existing_records:
            values_to_insert.append(tuple(row))
        else:
            #verificar si hay cambios en stock_splits
            existing_splits = existing_records[record][0]
            if existing_splits != row[1]:  #comparar el valor de stock_splits
                records_to_update.append(tuple(row))

    #totales
    rows_inserted = 0
    rows_updated = 0

    #insertar solo los nuevos registros
    if values_to_insert:
        cursor.executemany(insert_query, values_to_insert)
        rows_inserted = cursor.rowcount

    #actualizar registros existentes que han cambiado
    if records_to_update:
        cursor.executemany(insert_query, records_to_update)
        rows_updated = cursor.rowcount

    db.commit()

    #resultados finales
    print(f"Total de registros existentes: {len(existing_records)}")
    print(f"Añadidas: {rows_inserted} filas nuevas")
    print(f"Actualizadas: {rows_updated} filas")
    
    cursor.close()
    db.close()

insert_to_splits(data=data, table=table, database=database)

Total de registros existentes: 31
Añadidas: 0 filas nuevas
Actualizadas: 0 filas


In [7]:
#función para actualizar tabla balance_sheet
data = pd.read_csv("balance_sheet_20241031.csv").replace({np.nan: None}).values
database = "yfinance_stocks"
table = "balance_sheet"

def insert_to_balance_sheet(data, table, database, host="localhost", user="root", password="Hackaboss_2024"):
    db = mysql.connector.connect(host=host, user=user, password=password, database=database)
    cursor = db.cursor()

    #consultas de inserción
    insert_query = f"""
        INSERT INTO {table} (date, ticker, ordinary_shares_number, net_debt, total_assets, current_liabilities, stockholders_equity, extraction_timestamp) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            ordinary_shares_number = IF(VALUES(ordinary_shares_number) != ordinary_shares_number, VALUES(ordinary_shares_number), ordinary_shares_number),
            net_debt = IF(VALUES(net_debt) != net_debt, VALUES(net_debt), net_debt),
            total_assets = IF(VALUES(total_assets) != total_assets, VALUES(total_assets), total_assets),
            current_liabilities = IF(VALUES(current_liabilities) != current_liabilities, VALUES(current_liabilities), current_liabilities),
            stockholders_equity = IF(VALUES(stockholders_equity) != stockholders_equity, VALUES(stockholders_equity), stockholders_equity),
            extraction_timestamp = CASE 
                WHEN VALUES(ordinary_shares_number) != ordinary_shares_number OR 
                     VALUES(net_debt) != net_debt OR 
                     VALUES(total_assets) != total_assets OR 
                     VALUES(current_liabilities) != current_liabilities OR 
                     VALUES(stockholders_equity) != stockholders_equity 
                THEN VALUES(extraction_timestamp) 
                ELSE extraction_timestamp 
            END
    """

    #obtener registros existentes
    existing_records = {}
    cursor.execute(f"SELECT date, ticker, ordinary_shares_number, net_debt, total_assets, current_liabilities, stockholders_equity, extraction_timestamp FROM {table}")
    for row in cursor.fetchall():
        existing_records[(row[0], row[1])] = row[2:]  #(date, ticker) : (ordinary_shares_number, net_debt, ...)

    #filtrado de nuevos registros
    values_to_insert = []
    records_to_update = []
    for row in data:
        record = (row[0], row[1])  #campos date y ticker
        if record not in existing_records:
            values_to_insert.append(tuple(row))
        else:
            #verificar si hay cambios en los otros campos
            existing_values = existing_records[record][:5]  #excluir extraction_timestamp
            if existing_values != tuple(row[2:7]):  #comparar todos los valores excepto extraction_timestamp
                records_to_update.append(tuple(row))

    #totales
    rows_inserted = 0
    rows_updated = 0

    #insertar solo los nuevos registros
    if values_to_insert:
        cursor.executemany(insert_query, values_to_insert)
        rows_inserted = cursor.rowcount

    #actualizar registros existentes que han cambiado
    if records_to_update:
        cursor.executemany(insert_query, records_to_update)
        rows_updated = cursor.rowcount

    db.commit()

    #resultados finales
    print(f"Total de registros existentes: {len(existing_records)}")
    print(f"Añadidas: {rows_inserted} filas nuevas")
    print(f"Actualizadas: {rows_updated} filas")
    
    cursor.close()
    db.close()

insert_to_balance_sheet(data=data, table=table, database=database)

Total de registros existentes: 23
Añadidas: 0 filas nuevas
Actualizadas: 0 filas


In [8]:
#función para actualizar tabla income_statement
data = pd.read_csv("income_statement_20241031.csv").replace({np.nan: None}).values
database = "yfinance_stocks"
table = "income_statement"

def insert_to_income_statement(data, table, database, host="localhost", user="root", password="Hackaboss_2024"):
    db = mysql.connector.connect(host=host, user=user, password=password, database=database)
    cursor = db.cursor()

    #consultas de inserción
    insert_query = f"""
        INSERT INTO {table} (date, ticker, ebitda, total_revenue, net_income, operating_income, diluted_eps, extraction_timestamp)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            ebitda = IF(VALUES(ebitda) != ebitda, VALUES(ebitda), ebitda),
            total_revenue = IF(VALUES(total_revenue) != total_revenue, VALUES(total_revenue), total_revenue),
            net_income = IF(VALUES(net_income) != net_income, VALUES(net_income), net_income),
            operating_income = IF(VALUES(operating_income) != operating_income, VALUES(operating_income), operating_income),
            diluted_eps = IF(VALUES(diluted_eps) != diluted_eps, VALUES(diluted_eps), diluted_eps),
            extraction_timestamp = CASE 
                WHEN VALUES(ebitda) != ebitda OR
                     VALUES(total_revenue) != total_revenue OR
                     VALUES(net_income) != net_income OR
                     VALUES(operating_income) != operating_income OR
                     VALUES(diluted_eps) != diluted_eps 
                THEN VALUES(extraction_timestamp) 
                ELSE extraction_timestamp 
            END
    """

    #obtener registros existentes
    existing_records = {}
    cursor.execute(f"SELECT date, ticker, ebitda, total_revenue, net_income, operating_income, diluted_eps, extraction_timestamp FROM {table}")
    for row in cursor.fetchall():
        existing_records[(row[0], row[1])] = row[2:]  #(date, ticker) : (ebitda, total_revenue, ...)

    #filtrado de nuevos registros
    values_to_insert = []
    records_to_update = []
    for row in data:
        record = (row[0], row[1])  #campos date y ticker
        if record not in existing_records:
            values_to_insert.append(tuple(row))
        else:
            #verificar si hay cambios en los otros campos
            existing_values = existing_records[record]
            if existing_values != tuple(row[2:7]):  #comparar todos los valores excepto ticker extraction_timestamp
                records_to_update.append(tuple(row))


    rows_inserted = 0
    rows_updated = 0

    #insertar solo los nuevos registros
    if values_to_insert:
        cursor.executemany(insert_query, values_to_insert)
        rows_inserted = cursor.rowcount

    #actualizar registros existentes que han cambiado
    if records_to_update:
        cursor.executemany(insert_query, records_to_update)
        rows_updated = cursor.rowcount

    db.commit()

    #resultados finales
    print(f"Total de registros existentes: {len(existing_records)}")
    print(f"Añadidas: {rows_inserted} filas nuevas")
    print(f"Actualizadas: {rows_updated} filas")
    
    cursor.close()
    db.close()

insert_to_income_statement(data=data, table=table, database=database)

Total de registros existentes: 22
Añadidas: 0 filas nuevas
Actualizadas: 0 filas


In [9]:
#función para actualizar tabla cashflow
data = pd.read_csv("cashflow_20241031.csv").replace({np.nan: None}).values
database = "yfinance_stocks"
table = "cashflow"

def insert_to_cashflow(data, table, database, host="localhost", user="root", password="Hackaboss_2024"):
    db = mysql.connector.connect(host=host, user=user, password=password, database=database)
    cursor = db.cursor()

    #consultas de inserción
    insert_query = f"""
        INSERT INTO {table} (date, ticker, operating_cash_flow, capital_expenditure, free_cash_flow, extraction_timestamp)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            operating_cash_flow = IF(VALUES(operating_cash_flow) != operating_cash_flow, VALUES(operating_cash_flow), operating_cash_flow),
            capital_expenditure = IF(VALUES(capital_expenditure) != capital_expenditure, VALUES(capital_expenditure), capital_expenditure),
            free_cash_flow = IF(VALUES(free_cash_flow) != free_cash_flow, VALUES(free_cash_flow), free_cash_flow),
            extraction_timestamp = CASE 
                WHEN VALUES(operating_cash_flow) != operating_cash_flow OR 
                     VALUES(capital_expenditure) != capital_expenditure OR 
                     VALUES(free_cash_flow) != free_cash_flow THEN VALUES(extraction_timestamp)
                ELSE extraction_timestamp
            END
    """

    #obtener registros existentes
    existing_records = {}
    cursor.execute(f"SELECT date, ticker, operating_cash_flow, capital_expenditure, free_cash_flow, extraction_timestamp FROM {table}")
    for row in cursor.fetchall():
        existing_records[(row[0], row[1])] = row[2:]  #(date, ticker) : (operating_cash_flow, capital_expenditure, free_cash_flow, extraction_timestamp)

    #filtrado de nuevos registros
    values_to_insert = []
    records_to_update = []
    for row in data:
        record = (row[0], row[1])  #campos date y ticker
        if record not in existing_records:
            values_to_insert.append(tuple(row))
        else:
            #verificar si hay cambios en los otros campos
            existing_values = existing_records[record]
            if existing_values != tuple(row[2:5]):  #comparar los valores 
                records_to_update.append(tuple(row))

    #totales
    rows_inserted = 0
    rows_updated = 0

    #insertar solo los nuevos registros
    if values_to_insert:
        cursor.executemany(insert_query, values_to_insert)
        rows_inserted = cursor.rowcount

    #actualizar registros existentes que han cambiado
    if records_to_update:
        cursor.executemany(insert_query, records_to_update)
        rows_updated = cursor.rowcount
        
    db.commit()

    #resultados finales
    print(f"Total de registros existentes: {len(existing_records)}")
    print(f"Añadidas: {rows_inserted} filas nuevas")
    print(f"Actualizadas: {rows_updated} filas")

    cursor.close()
    db.close()

insert_to_cashflow(data=data, table=table, database=database)

Total de registros existentes: 24
Añadidas: 0 filas nuevas
Actualizadas: 0 filas


In [10]:
#función para actualizar tabla upgrades_downgrades
data = pd.read_csv("upgrades_downgrades_20241030.csv").replace({np.nan: None}).values
database = "yfinance_stocks"
table = "upgrades_downgrades"

def insert_to_upgrades_downgrades(data, table, database, host="localhost", user="root", password="Hackaboss_2024"):
    db = mysql.connector.connect(host=host, user=user, password=password, database=database)
    cursor = db.cursor()

    #consultas de inserción
    insert_query = f"""
        INSERT INTO {table} (date, firm, to_grade, from_grade, action, ticker, extraction_timestamp)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            to_grade = IF(VALUES(to_grade) != to_grade, VALUES(to_grade), to_grade),
            from_grade = IF(VALUES(from_grade) != from_grade, VALUES(from_grade), from_grade),
            action = IF(VALUES(action) != action, VALUES(action), action),
            extraction_timestamp = CASE 
                WHEN VALUES(to_grade) != to_grade OR 
                     VALUES(from_grade) != from_grade OR 
                     VALUES(action) != action 
                THEN VALUES(extraction_timestamp)
                ELSE extraction_timestamp
            END
    """

    #obtener registros existentes
    existing_records = {}
    cursor.execute(f"SELECT date, firm, ticker, to_grade, from_grade, action, extraction_timestamp FROM {table}")
    for row in cursor.fetchall():
        existing_records[(row[0], row[1], row[2])] = row[3:6]  #(date, firm, ticker) : (to_grade, from_grade, action)

    #filtrado de nuevos registros
    values_to_insert = []
    records_to_update = []
    for row in data:
        record_key = (row[0], row[1], row[5])  #clave (date, firm, ticker)
        if record_key not in existing_records:
            values_to_insert.append(tuple(row))
        else:
            #verificar si hay cambios en los campos
            existing_values = existing_records[record_key]
            if existing_values != tuple(row[2:5]):  #comparar to_grade, from_grade, action
                records_to_update.append(tuple(row))

    #totales
    rows_inserted = 0
    rows_updated = 0

    #insertar solo los nuevos registros
    if values_to_insert:
        cursor.executemany(insert_query, values_to_insert)
        rows_inserted = cursor.rowcount

    #actualizar registros existentes que han cambiado
    if records_to_update:
        cursor.executemany(insert_query, records_to_update)
        rows_updated = cursor.rowcount

    db.commit()

    #resultados finales
    print(f"Total de registros existentes: {len(existing_records)}")
    print(f"Añadidas: {rows_inserted} filas nuevas")
    print(f"Actualizadas: {rows_updated} filas")

    cursor.close()
    db.close()

insert_to_upgrades_downgrades(data=data, table=table, database=database)

Total de registros existentes: 3395
Añadidas: 0 filas nuevas
Actualizadas: 0 filas
